# Overview of the base class structure

`aeon` uses a core inheritance hierarchy of classes across the toolkit, with specialised sub classes in each module. The basic class hierarchy is shown in the following diagram.

<img src="img/aeon_uml_simple.drawio.png" alt="Basic class hierarchy">


## sklearn `BaseEstimator` and aeon `BaseAeonEstimator`

To make sense of this, we break it down from the top.
Everything inherits from sklearn `BaseEstimator`, which mainly handles the mechanisms for getting and setting parameters using the `set_params` and `get_params` methods. These methods are used when the estimators interact with other classes such as [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV), and is also used in aeon's `ComposableEstimatorMixin`, which we'll talk about later.

Then we have aeon's `BaseAeonEstimator` class. This class handles the following for all aeon's estimator:
- management of tags, setting, getting, interaction with sklearn's tags, etc.
- cloning and resetting of the estimator
- creation of test instances using test parameters specified by each estimators. For example, this is used to define fast-running estimator (e.g. a forest classifier with only 2 trees) for the CI/CD pipelines.

### A word on aeon's estimator tag system
Tags in aeon are used for various purposes, to display estimators capabilities in the documentations, to use specific tests based on each estimator's capabilities. You can check [all existing tags in aeon](https://github.com/aeon-toolkit/aeon/blob/main/aeon/utils/tags/_tags.py) and the [developer documentation on the testing framework](https://www.aeon-toolkit.org/en/stable/developer_guide/testing.html#) to know more about how we use tags.

## `BaseCollectionEstimator` and `BaseSeriesEstimator`

WIP

Is the base class for estimators that construct models on collections of time series. This includes classifiers, clusterers, regressors and collection transformers. It contains attributes and tags common to all these estimators, and protected methods to perform  checks and preprocessing common to all these estimators.


The subclasses of `BaseCollectionEstimator` are as follows they have similar interfaces, but they are not identical

## `BaseClassifier` (aeon.classification)

This is the base class for all classifiers. It uses the standard `fit`, `predict` and `predict_proba` structure from `sklearn`. `fit` and `predict` call the abstract methods `_fit` and `_predict` which are implemented in the subclass to define the classification algorithm. All of the common format checking and conversion is done using the following final methods defined in `BaseCollectionEstimator`.


## `BaseRegressor` (aeon.regression)

BaseRegressor has the same structure as `BaseClassifier`, although it has no `predict_proba` method. The tests on y are also different.

## `BaseClusterer` (aeon.clustering)

`BaseClusterer` also has `fit` and `predict`, but does not take input y. It does include `predict_proba`.

## `BaseCollectionTransformer` (aeon.transformations.collection)

The `BaseCollectionTransformer` was introduced to differentiate transformers that work on a single series to those that work on collections. Part of the motivation was to work around a lot of legacy code in `BaseTransformer` that performs a huge amount of conversion checks that is unnecessary for collections. Rather than `fit` and`predict` it implements `fit`, `transform` and `fit_transform`.

